In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

# 📁 Paths
data_path = r"C:\Users\sagni\Downloads\Sleep-Pattern-Detection\Sleep_health_and_lifestyle_dataset.csv"
model_save_path = r"C:\Users\sagni\Downloads\Sleep-Pattern-Detection\sleep_disorder_model.h5"

# 📦 Load dataset
df = pd.read_csv(data_path)
print("✅ Dataset loaded. Shape:", df.shape)

# 🧹 Preprocessing
# Map categorical columns to numeric
le_gender = LabelEncoder()
df['Gender'] = le_gender.fit_transform(df['Gender'])
le_bmi = LabelEncoder()
df['BMI Category'] = le_bmi.fit_transform(df['BMI Category'])
le_disorder = LabelEncoder()
df['Sleep Disorder'] = le_disorder.fit_transform(df['Sleep Disorder'])  # Target

# Extract Systolic & Diastolic from 'Blood Pressure'
df[['Systolic', 'Diastolic']] = df['Blood Pressure'].str.split('/', expand=True).astype(int)
df.drop(['Person ID', 'Occupation', 'Blood Pressure'], axis=1, inplace=True)

# Features and target
X = df.drop('Sleep Disorder', axis=1).values
y = df['Sleep Disorder'].values
y_cat = to_categorical(y)  # One-hot encode target

# 🔀 Split
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, random_state=42, stratify=y)

# ⚖️ Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 🤖 Build Neural Network
model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(3, activation='softmax')  # 3 classes: None, Apnea, Insomnia
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print("🧠 Model summary:")
model.summary()

# 🏋️‍♂️ Train
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                    epochs=100, batch_size=16, callbacks=[early_stop], verbose=1)

# 📊 Evaluate
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"✅ Test Accuracy: {acc:.4f}")

# 💾 Save model
model.save(model_save_path)
print(f"💾 Model saved to {model_save_path}")


✅ Dataset loaded. Shape: (374, 13)
🧠 Model summary:


C:\Users\sagni\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │             768 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 3)                   │              99 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,947 (11.51 KB)

 Trainable params: 2,947 (11.51 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.2453 - loss: 1.3540 - val_accuracy: 0.5867 - val_loss: 0.9867
Epoch 2/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5487 - loss: 0.9761 - val_accuracy: 0.9067 - val_loss: 0.7748
Epoch 3/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7703 - loss: 0.7829 - val_accuracy: 0.8933 - val_loss: 0.6061
Epoch 4/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8720 - loss: 0.5724 - val_accuracy: 0.8800 - val_loss: 0.4867
Epoch 5/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8241 - loss: 0.5934 - val_accuracy: 0.8800 - val_loss: 0.4226
Epoch 6/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8807 - loss: 0.4937 - val_accuracy: 0.8667 - val_loss: 0.3921
Epoch 7/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8822 - loss: 0.4378 - val_accuracy: 0.8667 - val_loss: 0.3761
Epoch 8/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9018 - loss: 0.4042 - val_accuracy: 0.8800 - 

✅ Test Accuracy: 0.8933
💾 Model saved to C:\Users\sagni\Downloads\Sleep-Pattern-Detection\sleep_disorder_model.h5
